In [1]:
import pandas as pd

# Caricamento del dataset
file_path = r'C:\Users\gallo\Downloads\archive\rating.csv'
ratings = pd.read_csv(file_path)

# Esplora i dati
print(ratings.head())
print(ratings.info())
print(len('movieId'))


   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB
None
7


In [2]:
# Raggruppa i film valutati da ciascun utente
user_movies = ratings.groupby('userId')['movieId'].apply(set)

# Visualizza un esempio
print(user_movies.head())



userId
1    {2, 3081, 4105, 1036, 5146, 541, 29, 32, 4128,...
2    {3, 260, 2948, 2951, 1544, 1673, 266, 908, 245...
3    {512, 1, 3072, 2053, 2054, 2567, 1544, 2571, 1...
4    {6, 519, 520, 10, 19, 531, 32, 420, 165, 548, ...
5    {2, 515, 260, 1028, 648, 11, 140, 141, 780, 10...
Name: movieId, dtype: object


In [3]:
import random
def minhash_signature(user_movies, num_hashes):
    """
    Genera una firma MinHash per un utente.
    - user_movies: Set di ID film valutati dall'utente.
    - num_hashes: Numero di funzioni hash da usare.
    """
    signature = []
    for i in range(num_hashes):
        # Crea una funzione hash randomica usando un seed unico
        random.seed(i)  # Cambia il comportamento del random
        a, b = random.randint(1, 100003), random.randint(0, 100003)
        hash_func = lambda x: (a * x + b) % 100003  # Più casualità
        
        # Applica la funzione hash al set dei film
        min_hash = min(hash_func(movie) for movie in user_movies)
        signature.append(min_hash)
    return signature


In [4]:
# Numero di funzioni hash
num_hashes = 10

# Genera le firme per tutti gli utenti
user_signatures = {user: minhash_signature(movies, num_hashes) for user, movies in user_movies.items()}

# Visualizza un esempio
for user, signature in list(user_signatures.items())[:5]:
    print(f"Utente {user}: {signature[:10]} (prime 10 hash)")


Utente 1: [330, 281, 314, 1225, 286, 677, 550, 564, 204, 1392] (prime 10 hash)
Utente 2: [1393, 729, 1066, 3363, 819, 280, 1007, 213, 2620, 1295] (prime 10 hash)
Utente 3: [131, 57, 504, 895, 2440, 677, 168, 747, 700, 454] (prime 10 hash)
Utente 4: [2304, 9222, 6476, 1304, 3782, 4501, 2847, 13449, 11321, 4734] (prime 10 hash)
Utente 5: [330, 2477, 2407, 1304, 987, 2585, 2847, 183, 2620, 799] (prime 10 hash)


In [5]:
def jaccard_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2)


In [6]:
def minhash_similarity(signature1, signature2):
    return sum(1 for i, j in zip(signature1, signature2) if i == j) / len(signature1)


In [7]:
# Seleziona due utenti a caso
user1, user2 = list(user_signatures.keys())[3:5]
signature1, signature2 = user_signatures[user1], user_signatures[user2]
movies1, movies2 = user_movies[user1], user_movies[user2]

# Calcola la similarità di Jaccard
real_similarity = jaccard_similarity(movies1, movies2)
print(f"Similarità reale di Jaccard tra utente {user1} e utente {user2}: {real_similarity:.2f}")

# Calcola la similarità MinHash
estimated_similarity = minhash_similarity(signature1, signature2)
print(f"Similarità stimata MinHash tra utente {user1} e utente {user2}: {estimated_similarity:.2f}")


Similarità reale di Jaccard tra utente 4 e utente 5: 0.15
Similarità stimata MinHash tra utente 4 e utente 5: 0.20


In [31]:
import pandas as pd
from itertools import combinations

# Percorso del file
file_path = r'C:\Users\gallo\Downloads\archive\rating.csv'

# Carica solo le prime 518 righe
ratings_subset = pd.read_csv(file_path, nrows=56301)

# Raggruppa i film valutati da ciascun utente in un dizionario
user_movies = ratings_subset.groupby('userId')['movieId'].apply(set).to_dict()

# Trova tutte le coppie di utenti
user_pairs = list(combinations(user_movies.keys(), 2))

# Calcola i film in comune per ogni coppia
results = []
for user1, user2 in user_pairs:
    # Intersezione
    common_movies = user_movies[user1] & user_movies[user2]
    num_common = len(common_movies)

    # Unione
    all_movies = user_movies[user1] | user_movies[user2]
    num_union = len(all_movies)

    # Rapporto (similarità di Jaccard)
    ratio = num_common / num_union if num_union > 0 else 0

    # Salva i risultati
    results.append((user1, user2, num_common, num_union, ratio))

# Crea un DataFrame con i risultati
results_df = pd.DataFrame(results, columns=['User1', 'User2', 'Intersection', 'Union', 'Ratio'])

# Visualizza i risultati
print(results_df)

       User1  User2  Intersection  Union     Ratio
0          1      2             9    227  0.039648
1          1      3            42    320  0.131250
2          1      4             3    200  0.015000
3          1      5            14    227  0.061674
4          1      6             2    197  0.010152
...      ...    ...           ...    ...       ...
83431    406    408             0    116  0.000000
83432    406    409            11    359  0.030641
83433    407    408             0    127  0.000000
83434    407    409            21    360  0.058333
83435    408    409             0    296  0.000000

[83436 rows x 5 columns]


In [32]:
import pandas as pd
import random
from itertools import combinations

# Percorso del file
file_path = r'C:\Users\gallo\Downloads\archive\rating.csv'

# Carica solo le prime 518 righe
ratings_subset = pd.read_csv(file_path, nrows=56301)

# Raggruppa i film valutati da ciascun utente in un dizionario
user_movies = ratings_subset.groupby('userId')['movieId'].apply(set).to_dict()

# Funzione MinHash per generare firme
def minhash_signature(user_movies, num_hashes):
    """
    Genera una firma MinHash per un utente.
    - user_movies: Set di ID film valutati dall'utente.
    - num_hashes: Numero di funzioni hash da usare.
    """
    signature = []
    for i in range(num_hashes):
        random.seed(i)  # Per ottenere risultati ripetibili
        a, b = random.randint(1, 100003), random.randint(0, 100003)
        hash_func = lambda x: (a * x + b) % 100003  # Funzione hash casuale
        min_hash = min(hash_func(movie) for movie in user_movies)
        signature.append(min_hash)
    return signature

# Genera le firme MinHash per ogni utente
num_hashes = 70  # Numero di funzioni hash
user_signatures = {user: minhash_signature(movies, num_hashes) for user, movies in user_movies.items()}

# Funzioni per similarità di Jaccard e MinHash
def jaccard_similarity(set1, set2):
    """Calcola la similarità di Jaccard tra due set."""
    return len(set1 & set2) / len(set1 | set2) if len(set1 | set2) > 0 else 0

def minhash_similarity(signature1, signature2):
    """Calcola la similarità stimata tramite MinHash."""
    return sum(1 for i, j in zip(signature1, signature2) if i == j) / len(signature1)

# Calcola intersezione, unione, Jaccard e MinHash per tutte le coppie
results = []
for user1, user2 in combinations(user_movies.keys(), 2):
    # Intersezione e unione
    common_movies = user_movies[user1] & user_movies[user2]
    all_movies = user_movies[user1] | user_movies[user2]

    num_common = len(common_movies)  # Intersezione
    num_union = len(all_movies)  # Unione

    # Similarità di Jaccard
    jaccard = jaccard_similarity(user_movies[user1], user_movies[user2])

    # Similarità stimata tramite MinHash
    minhash_sim = minhash_similarity(user_signatures[user1], user_signatures[user2])

    # Salva i risultati
    results.append((user1, user2, num_common, num_union, jaccard, minhash_sim))

# Crea un DataFrame con i risultati
results_df = pd.DataFrame(results, columns=['User1', 'User2', 'Intersection', 'Union', 'JaccardSimilarity', 'MinHashSimilarity'])

# Visualizza i risultati
print(results_df)

# Salva il risultato in un file CSV (opzionale)
results_df.to_csv(r'C:\Users\gallo\Downloads\user_similarity_with_minhash.csv', index=False)


       User1  User2  Intersection  Union  JaccardSimilarity  MinHashSimilarity
0          1      2             9    227           0.039648           0.014286
1          1      3            42    320           0.131250           0.071429
2          1      4             3    200           0.015000           0.014286
3          1      5            14    227           0.061674           0.042857
4          1      6             2    197           0.010152           0.000000
...      ...    ...           ...    ...                ...                ...
83431    406    408             0    116           0.000000           0.000000
83432    406    409            11    359           0.030641           0.042857
83433    407    408             0    127           0.000000           0.000000
83434    407    409            21    360           0.058333           0.042857
83435    408    409             0    296           0.000000           0.000000

[83436 rows x 6 columns]


1.2

In [33]:
import hashlib

def polynomial_hash(movie, coeffA, coeffB, i, prime):
    """Funzione hash polinomiale."""
    hash_value = 0
    for power, coeff in enumerate(coeffA[:i+1]):
        hash_value += coeff * (movie ** power)
    return (hash_value + coeffB[i]) % prime

def md5_hash(movie, seed):
    """Funzione hash basata su MD5."""
    hash_object = hashlib.md5(f"{movie}_{seed}".encode())
    return int(hash_object.hexdigest(), 16) % nextPrime


In [34]:
# Costanti per le funzioni hash
nextPrime = 100003
numHashes = 70
random.seed(42)
coeffA = [random.randint(1, nextPrime - 1) for _ in range(numHashes)]
coeffB = [random.randint(0, nextPrime - 1) for _ in range(numHashes)]

# Struttura per memorizzare le firme MinHash
minhash_signatures = {}

# Calcola le firme MinHash
for user, movies in user_movies.items():
    signature = {"linear": [], "polynomial": [], "md5": []}
    
    for i in range(numHashes):
        # Lineare (funzione hash attuale)
        minHashCode = nextPrime + 1
        for movie in movies:
            hashCode = (coeffA[i] * movie + coeffB[i]) % nextPrime
            minHashCode = min(minHashCode, hashCode)
        signature["linear"].append(minHashCode)
        
        # Polinomiale
        polyHashCode = nextPrime + 1
        for movie in movies:
            hashCode = polynomial_hash(movie, coeffA, coeffB, i, nextPrime)
            polyHashCode = min(polyHashCode, hashCode)
        signature["polynomial"].append(polyHashCode)
        
        # MD5
        md5HashCode = nextPrime + 1
        for movie in movies:
            hashCode = md5_hash(movie, i)
            md5HashCode = min(md5HashCode, hashCode)
        signature["md5"].append(md5HashCode)
    
    # Associa la firma all'utente
    minhash_signatures[user] = signature


In [35]:
results = []

# Calcola le similarità per ogni coppia di utenti e ogni tipo di firma
for user1, user2 in combinations(user_movies.keys(), 2):
    # Similarità di Jaccard
    jaccard = jaccard_similarity(user_movies[user1], user_movies[user2])
    
    # Similarità stimata per ogni tipo di firma
    for hash_type in ["linear", "polynomial", "md5"]:
        minhash_sim = minhash_similarity(
            minhash_signatures[user1][hash_type],
            minhash_signatures[user2][hash_type]
        )
        results.append((user1, user2, hash_type, jaccard, minhash_sim))

# Crea un DataFrame con i risultati
results_df = pd.DataFrame(results, columns=['User1', 'User2', 'HashType', 'JaccardSimilarity', 'MinHashSimilarity'])

# Visualizza i risultati
display(results_df)


,User1,User2,HashType,JaccardSimilarity,MinHashSimilarity
0,1,2,linear,0.039648,0.028571
1,1,2,polynomial,0.039648,0.014286
2,1,2,md5,0.039648,0.028571
3,1,3,linear,0.131250,0.114286
4,1,3,polynomial,0.131250,0.085714
...,...,...,...,...,...
250303,407,409,polynomial,0.058333,0.100000
250304,407,409,md5,0.058333,0.057143
250305,408,409,linear,0.000000,0.000000
250306,408,409,polynomial,0.000000,0.014286


In [36]:
# Calcolo del MAE senza reset_index
mae_results = results_df.groupby('HashType').apply(
    lambda x: abs(x['JaccardSimilarity'] - x['MinHashSimilarity']).mean()
)

# Creazione manuale del DataFrame
mae_results_df = pd.DataFrame({
    'HashType': mae_results.index,
    'MAE': mae_results.values
})

display(mae_results_df)

C:\Users\gallo\AppData\Local\Temp\ipykernel_7256\2840581765.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mae_results = results_df.groupby('HashType').apply(


,HashType,MAE
0,linear,0.016097
1,md5,0.016034
2,polynomial,0.018313


1.3.1

In [38]:
import hashlib
from collections import defaultdict

def create_buckets(user_signatures, num_bands, rows_per_band):
    """
    Crea i bucket dividendo le firme MinHash in bande e hashando ogni banda.
    
    Parameters:
    - user_signatures: Dizionario {user_id: MinHash_signature}.
    - num_bands: Numero di bande in cui dividere le firme MinHash.
    - rows_per_band: Numero di righe per banda.
    
    Returns:
    - buckets: Dizionario {bucket_id: lista_di_utenti}.
    """
    buckets = defaultdict(list)  # Bucket per raggruppare gli utenti

    # Ciclo su ogni utente e la sua firma
    for user_id, signature in user_signatures.items():
        # Dividi la firma MinHash in bande
        for band in range(num_bands):
            # Estrai le righe della banda corrente
            start_idx = band * rows_per_band
            end_idx = start_idx + rows_per_band
            band_rows = tuple(signature[start_idx:end_idx])

            # Hasha la banda per creare un bucket ID unico
            band_hash = hashlib.sha1(str(band_rows).encode()).hexdigest()

            # Aggiungi l'utente al bucket corrispondente
            buckets[band_hash].append(user_id)
    
    return buckets


In [ ]:
# Parametri per LSH
num_hashes = 15 # Numero di funzioni hash usate nelle firme MinHash
num_bands = 3  # Numero di bande
rows_per_band = num_hashes // num_bands  # Calcola il numero di righe per banda

# Crea i bucket utilizzando le firme MinHash
buckets = create_buckets(user_signatures, num_bands, rows_per_band)

# Stampa i bucket per verifica
for bucket_id, users in list(buckets.items())[270:300]:  # Mostra i primi 5 bucket
    print(f"Bucket {bucket_id}: {users}")

Bucket 6687d412679c2084b139ce61fe9cf447228f6742: [91]
Bucket b150d337434a9825976bd3535e27a819d0db0166: [91]
Bucket 980fd34f7e5d246bbfb4eb0b28f20eb4e7cbda64: [92]
Bucket 55e2e0404b5cd4919ae5d25cc8395367fad242a2: [92]
Bucket 5bc981c08cad688b6b8db72f06d7290bae521bb2: [92]
Bucket 55023557ea1c7cf27b34066fad6517786dc37ff2: [93]
Bucket f5850ba5086c9e9c2bcde6751730df08a92802c8: [93]
Bucket a1740993a7114ed017a385b00858afafd61d2369: [93]
Bucket db3b040215179dfd5f032382ab43160d1f1b5c5b: [94]
Bucket ef9a10e3855ce61f1936473980b0e8ef154040c5: [94]
Bucket 0ad494930ef86d3be11c19efe580fad321cec24c: [94]
Bucket 6d177adaf4cd2c5830c0cd008c306b454d96d0d3: [95]
Bucket cb7f9d8cc5c67277bc36fb23734beae5f2b2b5e7: [95]
Bucket 8ededf6b0e288b18db925401b8a98c889d15caa2: [95]
Bucket 74fc41d8f8e061db6da2489b4c599b0fa0c8ae20: [96]
Bucket bfdd5609a2244ffd93e81a8aa87a8f026bbc53a2: [96]
Bucket 12e1d4484b0e6a31731ab75cc84674f3e8a0a1d3: [96]
Bucket 8eae15240dafcb220ea49004b5338b68c286186a: [97]
Bucket e00a10309465a7e1ddaa6

In [74]:
# Debug per vedere le bande di un utente
for user_id, signature in user_signatures.items():
    for band in range(num_bands):
        start_idx = band * rows_per_band
        end_idx = start_idx + rows_per_band
        band_rows = tuple(signature[start_idx:end_idx])
        print(f"User: {user_id}, Band {band}: {band_rows}")


User: 1, Band 0: (330, 281, 314, 1225, 286)
User: 1, Band 1: (677, 550, 564, 204, 1392)
User: 1, Band 2: (76, 228, 1537, 493, 669)
User: 2, Band 0: (1393, 729, 1066, 3363, 819)
User: 2, Band 1: (280, 1007, 213, 2620, 1295)
User: 2, Band 2: (2512, 827, 4199, 3967, 95)
User: 3, Band 0: (131, 57, 504, 895, 2440)
User: 3, Band 1: (677, 168, 747, 700, 454)
User: 3, Band 2: (455, 2, 70, 85, 705)
User: 4, Band 0: (2304, 9222, 6476, 1304, 3782)
User: 4, Band 1: (4501, 2847, 13449, 11321, 4734)
User: 4, Band 2: (658, 4792, 3755, 3210, 1702)
User: 5, Band 0: (330, 2477, 2407, 1304, 987)
User: 5, Band 1: (2585, 2847, 183, 2620, 799)
User: 5, Band 2: (4067, 1145, 3755, 1929, 236)
User: 6, Band 0: (4159, 11552, 12729, 3332, 2161)
User: 6, Band 1: (4978, 10615, 4478, 3291, 1082)
User: 6, Band 2: (658, 5194, 5616, 3404, 2734)
User: 7, Band 0: (131, 124, 33, 895, 987)
User: 7, Band 1: (296, 116, 452, 339, 163)
User: 7, Band 2: (206, 2, 70, 85, 95)
User: 8, Band 0: (369, 595, 368, 3615, 987)
User: 8, B

In [59]:
# Verifica utenti nello stesso bucket
test_user = "User1"
similar_users = []

for bucket_id, users in buckets.items():
    if test_user in users:
        similar_users.extend([user for user in users if user != test_user])

# Rimuovi duplicati e stampa gli utenti simili
similar_users = list(set(similar_users))
print(f"Utenti simili a {test_user}: {similar_users}")


Utenti simili a User1: []


In [ ]:
for bucket_id, users in buckets.items():
    if len(users) > 1:
        print(f"Bucket {bucket_id} contains users: {users}")
        


Bucket bb8f5c535232e1bba2a01035a30b79f6ab3b332e contains users: [5, 26, 333]
Bucket 2922662bf92090b5495a3fd997e335cb0e3fe7c7 contains users: [5, 19, 126]
Bucket 06c4d615ddc5ec1e2b0aaeadd6fbbfbc62db3fbf contains users: [6, 126, 325]
Bucket 2f2238f32a3546a47d9df250704a3e519af28a74 contains users: [6, 354]
Bucket 3dddb8caef347ac56d08fcc9ab86288261daef0e contains users: [7, 24, 137]
Bucket c363081a43c286b4bec4a5b6097332f74ec26bc9 contains users: [7, 53, 54, 208, 245, 375]
Bucket efcdc3698080c6f26ad76ee31eb989e53999b1cd contains users: [8, 224, 244, 300]
Bucket 6583055da110ebd83db206242db9ed8a639de4ab contains users: [10, 55]
Bucket d39dc08e4d3d71fb8d798a8a96298ee411a4d6b5 contains users: [12, 15]
Bucket 5008135d8783dc14f86b392f5e515d59a286f560 contains users: [12, 242]
Bucket f801139e6bb2832b7e832898818b55f8189bd57c contains users: [12, 39]
Bucket 77f92b31e6c1d3a9809d4cc0264a18516168dd9e contains users: [13, 119, 120, 142, 249, 259, 296]
Bucket 5a645df9c3f4946f96044e597d5d85e75bfa57bb cont